In [1]:
import pandas as pd
import numpy as np
import os
from datetime import timedelta
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


def detect_host_pages_with_viz(input_path, output_dir='host_page_output', max_records_per_file=500000):
    """
    Phase 3.2: Host Page Detection (Memory-Optimized for 16M+ records)
    Host Page = ≥3 crawls + Max 10-day gap (NO percentile filtering)
    """
    
    os.makedirs(output_dir, exist_ok=True)
    
    print("📖 Loading crawl data...")
    dfs = []
    
    if os.path.isdir(input_path):
        files = [f for f in os.listdir(input_path) 
                if f.endswith(('.csv', '.xlsx', '.xls')) 
                and not f.startswith('~$')
                and not f.startswith('.')]
        
        if not files:
            print(f"❌ No CSV/Excel files found in folder: {input_path}")
            return None, None
            
        print(f"📁 Found {len(files)} file(s) in folder")
        
        for file in tqdm(files, desc="Reading files"):
            file_path = os.path.join(input_path, file)
            try:
                if file.endswith('.csv'):
                    temp_df = pd.read_csv(file_path, encoding='utf-8-sig', low_memory=False)
                else:
                    temp_df = pd.read_excel(file_path)
                dfs.append(temp_df)
            except Exception as e:
                print(f"⚠️ Error reading {file}: {str(e)[:100]}")
                continue
        
        if not dfs:
            print("❌ No data loaded from folder")
            return None, None
            
        print("   ├─ Concatenating DataFrames...")
        df = pd.concat(dfs, ignore_index=True)
        print(f"✅ Loaded {len(df):,} records from {len(dfs)} file(s)")
        
    else:
        if not os.path.exists(input_path):
            print(f"❌ File/folder not found: {input_path}")
            return None, None
            
        df = pd.read_excel(input_path) if input_path.endswith('.xlsx') else pd.read_csv(input_path)
        print(f"✅ Loaded {len(df):,} records from single file")
    
    # Memory optimization - UPDATED: Add 'status' column
    print("⚡ Optimizing memory...")
    progress_mem = widgets.IntProgress(value=0, min=0, max=100, description='Memory:', bar_style='info')
    display(progress_mem)
    
    progress_mem.value = 50
    df = df[['request_uri', 'time_iso8601', 'http_user_agent', 'status']].copy()  # ADDED status
    progress_mem.value = 100
    
    # Vectorized URL cleaning
    print("⚡ Processing URLs...")
    progress_url = widgets.IntProgress(value=0, min=0, max=100, description='URLs:', bar_style='info')
    display(progress_url)
    
    progress_url.value = 33
    df['timestamp'] = pd.to_datetime(df['time_iso8601'])
    progress_url.value = 66
    df['url_clean'] = df['request_uri'].str.split('?').str[0]
    df['url'] = 'https://www.alamy.com' + df['url_clean'].fillna('')
    progress_url.value = 100
    
    # Drop intermediate columns - UPDATED: Keep status
    df = df[['url', 'timestamp', 'http_user_agent', 'status']].copy()
    
    min_date, max_date = df['timestamp'].min(), df['timestamp'].max()
    period_days = (max_date - min_date).days + 1
    total_pages = df['url'].nunique()
    
    print(f"📅 Period: {min_date.date()} to {max_date.date()} ({period_days} days)")
    print(f"📊 Total unique pages: {total_pages:,}")
    
    print("\n⚙️ Calculating page statistics...")
    
    # ============================================================================
    # MEMORY-OPTIMIZED GROUPING (Split into stages)
    # ============================================================================
    
    # Step 1: First groupby - counts only (no lists, saves memory)
    print("   ├─ Step 1/6: Aggregating crawl counts (lightweight)...")
    progress_group1 = widgets.IntProgress(value=0, min=0, max=100, description='Counts:', bar_style='info')
    display(progress_group1)
    
    progress_group1.value = 30
    url_stats = df.groupby('url', sort=False).agg({
        'timestamp': ['count', 'min', 'max']
    }).reset_index()
    url_stats.columns = ['url', 'crawl_count', 'first_crawl', 'last_crawl']
    progress_group1.value = 100
    print(f"   │     └─ ✅ Aggregated {len(url_stats):,} unique URLs")
    
    # Step 2: Filter to only URLs with ≥3 crawls (reduces dataset before expensive operations)
    print("   ├─ Step 2/6: Pre-filtering URLs (≥3 crawls)...")
    progress_filter = widgets.IntProgress(value=0, min=0, max=100, description='Filtering:', bar_style='info')
    display(progress_filter)
    
    progress_filter.value = 50
    candidate_urls = url_stats[url_stats['crawl_count'] >= 3]['url'].tolist()
    df_candidates = df[df['url'].isin(candidate_urls)].copy()
    progress_filter.value = 100
    print(f"   │     └─ ✅ Filtered to {len(candidate_urls):,} candidates ({len(df_candidates):,} records)")
    
    # Step 3: Create timestamp lists ONLY for candidate URLs (much smaller dataset)
    print("   ├─ Step 3/6: Grouping timestamps (filtered set)...")
    progress_group2 = widgets.IntProgress(value=0, min=0, max=100, description='Timestamps:', bar_style='info')
    display(progress_group2)
    
    progress_group2.value = 30
    timestamp_groups = df_candidates.groupby('url', sort=False)['timestamp'].apply(list).reset_index()
    timestamp_groups.columns = ['url', 'timestamps']
    progress_group2.value = 100
    print(f"   │     └─ ✅ Created timestamp lists for {len(timestamp_groups):,} URLs")
    
    # Step 4: Get user agents
    print("   ├─ Step 4/6: Extracting user agents...")
    progress_ua = widgets.IntProgress(value=0, min=0, max=100, description='User Agents:', bar_style='info')
    display(progress_ua)
    
    progress_ua.value = 50
    user_agent_groups = df_candidates.groupby('url', sort=False)['http_user_agent'].apply(
        lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0]
    ).reset_index()
    user_agent_groups.columns = ['url', 'user_agent']
    progress_ua.value = 100
    print(f"   │     └─ ✅ Extracted user agents for {len(user_agent_groups):,} URLs")
    
    # Step 5: Extract status codes - NEW
    print("   ├─ Step 5/6: Extracting status codes...")
    progress_status = widgets.IntProgress(value=0, min=0, max=100, description='Status Codes:', bar_style='info')
    display(progress_status)
    
    progress_status.value = 50
    status_groups = df_candidates.groupby('url', sort=False)['status'].apply(
        lambda x: x.mode()[0] if len(x.mode()) > 0 else 200
    ).reset_index()
    status_groups.columns = ['url', 'status_code']
    progress_status.value = 100
    print(f"   │     └─ ✅ Extracted status codes for {len(status_groups):,} URLs")
    
    # Merge all stats
    url_groups = url_stats.merge(timestamp_groups, on='url', how='left')
    url_groups = url_groups.merge(user_agent_groups, on='url', how='left')
    url_groups = url_groups.merge(status_groups, on='url', how='left')
    url_groups['status_code'] = url_groups['status_code'].fillna(200)  # Default 200 for crawled pages
    
    url_groups['crawl_frequency'] = url_groups['crawl_count'] / period_days
    
    # Step 6: Host eligibility check (only on candidates with timestamps)
    print("   └─ Step 6/6: Checking host eligibility (≥3 crawls + max 10-day gap)...")
    
    def check_host_eligibility_fast(timestamps, min_crawls=3, max_gap_days=10):
        if timestamps is None or not isinstance(timestamps, list) or len(timestamps) < min_crawls:
            return False
        sorted_ts = sorted(timestamps)
        gaps = [(sorted_ts[i+1] - sorted_ts[i]).days for i in range(len(sorted_ts) - 1)]
        return all(gap <= max_gap_days for gap in gaps)
    
    tqdm.pandas(desc="   │     └─ Validating intervals")
    url_groups['meets_interval'] = url_groups['timestamps'].progress_apply(check_host_eligibility_fast)
    
    # Classification
    print("\n✅ Classifying host pages...")
    url_groups['is_host'] = (url_groups['crawl_count'] >= 3) & (url_groups['meets_interval'])
    
    # Keep authority_score for visualization ONLY
    url_groups['authority_score'] = url_groups['crawl_count']  # Simplified: just use crawl_count
    
    host_pages = url_groups[url_groups['is_host']].copy()
    host_pages = host_pages.sort_values('authority_score', ascending=False)
    
    print(f"   └─ ✅ Identified {len(host_pages):,} host pages ({len(host_pages)/total_pages*100:.2f}% of total)")
    
    # ============================================================================
    # ANALYZE GOOGLEBOT TYPES
    # ============================================================================
    print("\n🤖 Analyzing Googlebot types...")
    
    def extract_bot_type(user_agent):
        if pd.isna(user_agent):
            return 'Unknown'
        
        ua_str = str(user_agent).lower()
        
        if 'smartphone' in ua_str and 'googlebot' in ua_str:
            return 'Googlebot-Smartphone'
        elif 'googlebot-image' in ua_str:
            return 'Googlebot-Image'
        elif 'googlebot-video' in ua_str:
            return 'Googlebot-Video'
        elif 'googlebot-news' in ua_str:
            return 'Googlebot-News'
        elif 'adsbot' in ua_str:
            return 'AdsBot-Google'
        elif 'mediapartners' in ua_str:
            return 'Mediapartners-Google'
        elif 'mobile' in ua_str and 'googlebot' in ua_str:
            return 'Googlebot-Mobile-Legacy' 
        elif 'googlebot' in ua_str:
            return 'Googlebot-Desktop'
        elif 'google' in ua_str:
            return 'Other-Google-Bot'
        else:
            return 'Non-Google'
    
    df['bot_type'] = df['http_user_agent'].apply(extract_bot_type)
    bot_stats = df.groupby('bot_type')['url'].nunique().reset_index()
    bot_stats.columns = ['bot_type', 'unique_urls']
    bot_stats = bot_stats.sort_values('unique_urls', ascending=False)
    
    print(f"   └─ ✅ Analyzed {len(bot_stats)} bot types")
    
    # VISUALIZATION CODE (unchanged)
    print("\n📊 Generating visualizations...")
    progress_viz = widgets.IntProgress(value=0, min=0, max=100, description='Visuals:', bar_style='warning')
    display(progress_viz)
    
    progress_viz.value = 20
    df['date'] = df['timestamp'].dt.date
    host_urls = set(host_pages['url'].tolist())
    df['is_host_crawl'] = df['url'].isin(host_urls)
    
    progress_viz.value = 40
    daily_stats = df.groupby('date').agg({
        'url': 'nunique',
        'is_host_crawl': 'sum'
    }).reset_index()
    daily_stats.columns = ['date', 'total_pages_crawled', 'host_pages_crawled']
    progress_viz.value = 60
    
    output1 = widgets.Output()
    
    with output1:
        fig, ax = plt.subplots(2, 1, figsize=(14, 10))
        
        ax[0].plot(daily_stats['date'], daily_stats['total_pages_crawled'], 
                   label='Total Pages Crawled', linewidth=2, color='steelblue', alpha=0.7)
        ax[0].plot(daily_stats['date'], daily_stats['host_pages_crawled'], 
                   label='Host Pages Crawled', linewidth=2, color='orangered')
        ax[0].set_xlabel('Date', fontsize=12)
        ax[0].set_ylabel('Page Count', fontsize=12)
        ax[0].set_title('Host Pages vs Total Pages Crawled Over Time', fontsize=14, fontweight='bold')
        ax[0].legend(loc='upper left', fontsize=10)
        ax[0].grid(True, alpha=0.3)
        plt.setp(ax[0].xaxis.get_majorticklabels(), rotation=45)
        
        daily_stats['host_pct'] = (daily_stats['host_pages_crawled'] / daily_stats['total_pages_crawled']) * 100
        ax[1].fill_between(daily_stats['date'], daily_stats['host_pct'], 
                           color='mediumseagreen', alpha=0.5, label='Host Page %')
        ax[1].plot(daily_stats['date'], daily_stats['host_pct'], 
                   color='darkgreen', linewidth=2)
        ax[1].axhline(y=daily_stats['host_pct'].mean(), color='red', 
                      linestyle='--', label=f'Avg: {daily_stats["host_pct"].mean():.1f}%')
        ax[1].set_xlabel('Date', fontsize=12)
        ax[1].set_ylabel('Host Page %', fontsize=12)
        ax[1].set_title('Host Page Crawl Percentage Over Time', fontsize=14, fontweight='bold')
        ax[1].legend(loc='upper left', fontsize=10)
        ax[1].grid(True, alpha=0.3)
        plt.setp(ax[1].xaxis.get_majorticklabels(), rotation=45)
        
        plt.tight_layout()
        plt.show()
    
    output2 = widgets.Output()
    
    with output2:
        fig, ax = plt.subplots(1, 2, figsize=(16, 6))
        
        top_10 = host_pages.head(10)
        colors_gradient = plt.cm.Reds(np.linspace(0.4, 0.9, len(top_10)))
        
        ax[0].barh(range(len(top_10)), top_10['crawl_count'].values, color=colors_gradient)  # CHANGED to crawl_count
        ax[0].set_yticks(range(len(top_10)))
        ax[0].set_yticklabels([f"{i+1}. {url.split('/')[-1][:30]}..." for i, url in enumerate(top_10['url'].values)], fontsize=9)
        ax[0].set_xlabel('Crawl Count', fontsize=12, fontweight='bold')  # CHANGED label
        ax[0].set_title('Top 10 Host Pages by Crawl Count', fontsize=14, fontweight='bold')  # CHANGED title
        ax[0].invert_yaxis()
        ax[0].grid(axis='x', alpha=0.3)
        
        for i, v in enumerate(top_10['crawl_count'].values):  # CHANGED to crawl_count
            ax[0].text(v + max(top_10['crawl_count'])*0.01, i, f'{v:.0f}',   # CHANGED format
                      va='center', fontsize=9, fontweight='bold')
        
        colors_bot = plt.cm.Blues(np.linspace(0.4, 0.9, len(bot_stats)))
        
        ax[1].bar(range(len(bot_stats)), bot_stats['unique_urls'].values, color=colors_bot, edgecolor='black', linewidth=1)
        ax[1].set_xticks(range(len(bot_stats)))
        ax[1].set_xticklabels(bot_stats['bot_type'].values, rotation=45, ha='right', fontsize=9)
        ax[1].set_ylabel('Unique URLs Crawled', fontsize=12, fontweight='bold')
        ax[1].set_title('Googlebot Types - URL Coverage', fontsize=14, fontweight='bold')
        ax[1].grid(axis='y', alpha=0.3)
        
        for i, v in enumerate(bot_stats['unique_urls'].values):
            ax[1].text(i, v + max(bot_stats['unique_urls'])*0.01, f'{v:,}', 
                      ha='center', va='bottom', fontsize=9, fontweight='bold')
        
        plt.tight_layout()
        plt.show()
    
    progress_viz.value = 80
    
    output3 = widgets.Output()
    
    with output3:
        fig, ax = plt.subplots(1, 1, figsize=(10, 6))
        
        categories = ['Host Pages', 'Regular Pages']
        counts = [len(host_pages), total_pages - len(host_pages)]
        colors = ['#ff6b6b', '#4ecdc4']
        
        ax.bar(categories, counts, color=colors, edgecolor='black', linewidth=1.5, width=0.6)
        ax.set_ylabel('Page Count', fontsize=12, fontweight='bold')
        ax.set_title('Host Pages vs Regular Pages Distribution', fontsize=14, fontweight='bold')
        ax.text(0, counts[0] + max(counts)*0.02, f'{counts[0]:,}', 
                   ha='center', fontsize=12, fontweight='bold')
        ax.text(1, counts[1] + max(counts)*0.02, f'{counts[1]:,}', 
                   ha='center', fontsize=12, fontweight='bold')
        ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    progress_viz.value = 100
    print("   └─ ✅ Visualizations complete")
    
    # SAVE OUTPUT - UPDATED
    print("\n💾 Saving output files...")
    progress_save = widgets.IntProgress(value=0, min=0, max=100, description='Saving:', bar_style='success')
    display(progress_save)
    
    # UPDATED: Remove authority_score, add status_code
    output_cols = ['url', 'crawl_count', 'crawl_frequency', 'user_agent', 'status_code']
    
    if len(host_pages) <= max_records_per_file:
        progress_save.value = 50
        output_path = os.path.join(output_dir, 'host_pages.csv')
        host_pages[output_cols].to_csv(output_path, index=False, encoding='utf-8-sig')
        progress_save.value = 100
        print(f"   └─ Saved: {output_path}")
    else:
        num_parts = (len(host_pages) // max_records_per_file) + 1
        print(f"   ├─ Splitting into {num_parts} parts...")
        for i in tqdm(range(num_parts), desc="   │  └─ Writing files"):
            start_idx = i * max_records_per_file
            end_idx = min((i + 1) * max_records_per_file, len(host_pages))
            part_df = host_pages.iloc[start_idx:end_idx]
            output_path = os.path.join(output_dir, f'host_pages_part{i+1}.csv')
            part_df[output_cols].to_csv(output_path, index=False, encoding='utf-8-sig')
            progress_save.value = int((i+1) / num_parts * 100)
        print(f"   └─ Saved {num_parts} files")
    
    print("\n" + "="*80)
    print("📊 PHASE 3.2 COMPLETE: HOST PAGE DETECTION")
    print("="*80)
    print(f"✅ Host Page Criteria: ≥3 crawls + max 10-day gap")
    print(f"✅ Output: url, crawl_count, crawl_frequency, user_agent, status_code")
    print(f"✅ Analyzed {len(bot_stats)} different bot types")
    print("="*80)
    
    display(output1)
    display(output2)
    display(output3)
    
    return host_pages, daily_stats, bot_stats


# Execution widgets
file_input = widgets.Text(
    value='Processready-test.xlsx',
    placeholder='Enter file or folder path',
    description='Input Path:',
    style={'description_width': 'initial'}
)

run_button = widgets.Button(
    description='Run Analysis',
    button_style='success',
    icon='check'
)

output_area = widgets.Output()

def on_run_clicked(b):
    with output_area:
        clear_output()
        detect_host_pages_with_viz(file_input.value)

run_button.on_click(on_run_clicked)

display(widgets.VBox([file_input, run_button, output_area]))
